In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/기타/4dblock/4dblock.zip

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2
import glob

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import KFold

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
import torchvision
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
print(torch.__version__)
print(torchvision.__version__)

1.13.1+cu116
0.14.1+cu116


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':15,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
df = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [ ]:
# path_list = []
# for arg in test["img_path"]:
#     path_list.append("./test_rmbg"+arg[6:])

In [ ]:
# test["img_path"] = path_list
# test.head()

In [ ]:
test.head()

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg


In [ ]:
df.head()

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,./train/TRAIN_00000.jpg,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,./train/TRAIN_00001.jpg,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,./train/TRAIN_00002.jpg,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,./train/TRAIN_00003.jpg,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,./train/TRAIN_00004.jpg,1,0,0,0,0,0,0,0,0,0


In [ ]:
df.iloc[:,2:]

,A,B,C,D,E,F,G,H,I,J
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
32989,1,0,0,0,1,0,1,1,1,1
32990,1,0,0,0,0,1,1,1,1,1
32991,1,0,0,0,0,1,1,1,1,1
32992,1,0,0,0,0,1,1,1,1,1


특정 클래스이면서, 다른 클래스에 속할 수 있다.

In [ ]:
# def add_img_info(dataframe):
#     width  = []
#     height = []
#     size   = []
#     for idx, d in enumerate(dataframe['img_path']):
#         path = os.path.join("/kaggle/input/4dblock-data/", d)
#         img = cv2.imread(path)
#         h, w, c = img.shape
#         height.append(h)
#         width.append(w)
#         size.append(h*w)
#     dataframe['width'] = width
#     dataframe['height'] = height
#     dataframe['img_size'] = size
#     return dataframe

In [ ]:
# train_img_df = add_img_info(train)
# test_img_df = add_img_info(test)

In [ ]:
# train_img_df.head()

In [ ]:
# test_img_df.head()

In [ ]:
back_list = []
for i in range(5):
    back_image = cv2.imread(str(i)+".jpeg")
    back_image = cv2.cvtColor(back_image, cv2.COLOR_BGR2RGB)
    back_image = cv2.resize(back_image, (224,224))
    back_list.append(back_image)

In [ ]:
def show_image(Image, Picture_Info):
    plt.figure(figsize=(7,7))
    plt.imshow(Image)
    plt.title(Picture_Info)
    plt.show()

def show_only_object(Image):
    # convert to graky
    rows, cols = Image.shape
    crow,ccol = rows//2 , cols//2
    f = np.fft.fft2(Image)
    fshift = np.fft.fftshift(f)
    fshift[crow-30:crow+31, ccol-30:ccol+31] = 0
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.real(img_back)

    
    return img_back
    
def show_img(dataframe):
    for idx, d in enumerate(dataframe['img_path']):
        path = os.path.join("./", d)
        img = cv2.imread(path)
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        dft = np.fft.fft2(img)
        dft = np.fft.fftshift(dft)
        print(dft.shape)
        show_image(img, dataframe["id"][idx])
        break
        
# https://tolovefeels.tistory.com/100
def show_only_img(dataframe):
    for idx, d in enumerate(dataframe['img_path']):
        j = idx%5
        path = os.path.join("./", d)
        img = cv2.imread(path) 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)

        # 변환 graky
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 임계값 조절
        mask = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)[1]

        # mask
        mask = 255 - mask

        # morphology 적용
        # borderconstant 사용
        kernel = np.ones((3,3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # anti-alias the mask
        # blur alpha channel
        mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

        # linear stretch so that 127.5 goes to 0, but 255 stays 255
        mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)

        # put mask into alpha channel
        result = img.copy()
        # result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
        print(mask, mask.shape)
        result[mask==0] = 0

        # put background image
        back_image = back_list[j]
        back_image[mask!=0] = 0
        # print(result.shape, back_image.shape)
        dst = np.where(result>250, back_image, result)
        result += back_image
        show_image(result,"hi")
        if idx==2:
            break

In [ ]:
# https://tolovefeels.tistory.com/100
def create_image(dataframe):
    for idx, d in enumerate(dataframe['img_path']):
        j = idx%5
        path = os.path.join("./", d)
        img = cv2.imread(path) 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)

        # 변환 graky
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 임계값 조절
        mask = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)[1]

        # mask
        mask = 255 - mask

        # morphology 적용
        # borderconstant 사용
        kernel = np.ones((3,3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # anti-alias the mask
        # blur alpha channel
        mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

        # linear stretch so that 127.5 goes to 0, but 255 stays 255
        mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)

        # put mask into alpha channel
        result = img.copy()
        # result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
        # print(mask, mask.shape)
        result[mask==0] = 0

        # put background image
        back_image = back_list[j]
        back_image[mask!=0] = 0
        result += back_image
        # show_image(result,"hi")
        # print(d[7:])
        output_path = os.path.join("/content/drive/MyDrive/기타/4dblock/train_back/", d[7:])
        cv2.imwrite(output_path, np.array(result))
        # plt.imshow(np.array(result))

In [ ]:
# show_img(df)

In [ ]:
# show_img(test)

In [ ]:
# show_only_img(df)

In [ ]:
# show_only_img(test)

In [ ]:
df = df.sample(frac=1)
train_len = int(len(df) * 0.8)

In [ ]:
# df["img_path"][0][7:]

In [ ]:
# path_list = []
# for arg in df["img_path"]:
#     path_list.append("./train_back"+arg[7:])

In [ ]:
# df["img_path"] = path_list
# df.head()

In [ ]:
train = df[:train_len]
val = df[train_len:]

In [ ]:
# def get_labels(df):
#     return df.iloc[:,2:].values

In [ ]:
# train_labels = get_labels(train)
# val_labels = get_labels(val)

In [ ]:
train_A = train["A"].values # sentence type
train_B = train["B"].values # sentence polarity
train_C = train["C"].values # sentence tense
train_D = train["D"].values # sentence certainty
train_E = train["E"].values # sentence type
train_F = train["F"].values # sentence polarity
train_G = train["G"].values # sentence tense
train_H = train["H"].values # sentence certainty
train_I = train["I"].values # sentence type
train_J = train["J"].values # sentence polarity

train_labels = {
    'A' : train_A,
    'B' : train_B,
    'C' : train_C,
    'D' : train_D,
    'E' : train_E,
    'F' : train_F,
    'G' : train_G,
    'H' : train_H,
    'I' : train_I,
    'J' : train_J
}

In [ ]:
val_A = val["A"].values # sentence type
val_B = val["B"].values # sentence polarity
val_C = val["C"].values # sentence tense
val_D = val["D"].values # sentence certainty
val_E = val["E"].values # sentence type
val_F = val["F"].values # sentence polarity
val_G = val["G"].values # sentence tense
val_H = val["H"].values # sentence certainty
val_I = val["I"].values # sentence type
val_J = val["J"].values # sentence polarity

val_labels = {
    'A' : val_A,
    'B' : val_B,
    'C' : val_C,
    'D' : val_D,
    'E' : val_E,
    'F' : val_F,
    'G' : val_G,
    'H' : val_H,
    'I' : val_I,
    'J' : val_J
}

In [ ]:
train_labels

{'A': array([0, 1, 0, ..., 0, 1, 0]),
 'B': array([1, 0, 1, ..., 1, 1, 1]),
 'C': array([1, 1, 1, ..., 0, 0, 1]),
 'D': array([0, 1, 1, ..., 0, 0, 0]),
 'E': array([1, 1, 0, ..., 1, 0, 1]),
 'F': array([0, 0, 0, ..., 1, 0, 1]),
 'G': array([0, 1, 0, ..., 0, 0, 0]),
 'H': array([0, 0, 0, ..., 1, 1, 0]),
 'I': array([0, 0, 0, ..., 0, 1, 1]),
 'J': array([0, 0, 0, ..., 1, 1, 1])}

In [ ]:
# train_labels = [list(x) for x in zip(*train_labels)] 
# val_labels = [list(x) for x in zip(*val_labels)] 

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        img_path = os.path.join("./", img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_CUBIC)
        
        j = index%5
        # 변환 graky
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # 임계값 조절
        mask = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)[1]

        # mask
        mask = 255 - mask

        # morphology 적용
        # borderconstant 사용
        kernel = np.ones((3,3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # anti-alias the mask
        # blur alpha channel
        mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

        # linear stretch so that 127.5 goes to 0, but 255 stays 255
        mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)

        # put mask into alpha channel
        result = image.copy()
        result[mask==0] = 0

        # put background image
        back_image = back_list[j]
        back_image[mask!=0] = 0
        result += back_image
        image = result

        if self.transforms is not None:
            image = self.transforms(image=image)['image']
#             image = np.fft.fft2(image)
#             image = np.fft.fftshift(image)
            
        if self.label_list is not None:
            #label = torch.FloatTensor(self.label_list[index])
            
            labels_A = self.label_list["A"][index]
            labels_B = self.label_list["B"][index]
            labels_C = self.label_list["C"][index]
            labels_D = self.label_list["D"][index]
            labels_E = self.label_list["E"][index]
            labels_F = self.label_list["F"][index]
            labels_G = self.label_list["G"][index]
            labels_H = self.label_list["H"][index]
            labels_I = self.label_list["I"][index]
            labels_J = self.label_list["J"][index]
            
            return image,labels_A,labels_B,labels_C,labels_D,labels_E,labels_F,labels_G,labels_H,labels_I,labels_J
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=10):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_v2_m(pretrained=True)
        self.my_dense = nn.Sequential(
            nn.Linear(1000,224),
            nn.LeakyReLU(),
        )
        # self.classifier = nn.Linear(224,num_classes)
        
        self.type_classifierA = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=224, out_features=1),
        )
        self.type_classifierB = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=224, out_features=1),
        )
        self.type_classifierC = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=224, out_features=1),
        )
        self.type_classifierD = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=224, out_features=1),
        )
        self.type_classifierE = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=224, out_features=1),
        )
        self.type_classifierF = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=224, out_features=1),
        )
        self.type_classifierG = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=224, out_features=1),
        )
        self.type_classifierH = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=224, out_features=1),
        )
        self.type_classifierI = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=224, out_features=1),
        )
        self.type_classifierJ = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=224, out_features=1),
        )
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.my_dense(x)
        # x = F.sigmoid(self.classifier(x))
        A = F.sigmoid(self.type_classifierA(x))
        B = F.sigmoid(self.type_classifierB(x))
        C = F.sigmoid(self.type_classifierC(x))
        D = F.sigmoid(self.type_classifierD(x))
        E = F.sigmoid(self.type_classifierE(x))
        F_ = F.sigmoid(self.type_classifierF(x))
        G = F.sigmoid(self.type_classifierG(x))
        H = F.sigmoid(self.type_classifierH(x))
        I = F.sigmoid(self.type_classifierI(x))
        J = F.sigmoid(self.type_classifierJ(x))
        return A,B,C,D,E,F_,G,H,I,J

In [ ]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                            A.HorizontalFlip(p = 1),
                            A.IAAAffine(shear=15, p=0.5, mode='constant'),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=1)

val_dataset = CustomDataset(val['img_path'].values, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=1)

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    # best_val_acc = 0
    best_val_loss = 99999
    best_model = None
    t_loss = []
    val_loss = []
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels_A,labels_B,labels_C,labels_D,labels_E,labels_F,labels_G,labels_H,labels_I,labels_J in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            # labels = labels.to(device)
            labels_A = labels_A.to(device)
            labels_B = labels_B.to(device)
            labels_C = labels_C.to(device)
            labels_D = labels_D.to(device)
            labels_E = labels_E.to(device)
            labels_F = labels_F.to(device)
            labels_G = labels_G.to(device)
            labels_H = labels_H.to(device)
            labels_I = labels_I.to(device)
            labels_J = labels_J.to(device)
            
            optimizer.zero_grad()
            
            A,B,C,D,E,F,G,H,I,J = model(imgs)
            # loss = criterion(output, labels)
            loss =  0.1 * criterion(A.to(torch.float32), labels_A.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(B.to(torch.float32), labels_B.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(C.to(torch.float32), labels_C.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(D.to(torch.float32), labels_D.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(E.to(torch.float32), labels_E.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(F.to(torch.float32), labels_F.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(G.to(torch.float32), labels_G.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(H.to(torch.float32), labels_H.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(I.to(torch.float32), labels_I.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(J.to(torch.float32), labels_J.view([-1, 1]).to(torch.float32))

            loss.backward()
            
            # for TPU
            optimizer.step()
            # xm.optimizer_step(optimizer, barrier=True)
            
            train_loss.append(loss.item())
                    
        _val_loss = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')

        # _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        # _train_loss = np.mean(train_loss)
        # print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        t_loss.append(_train_loss)
        val_loss.append(_val_loss)
        
        if scheduler is not None:
            scheduler.step(_val_loss)
            
        if best_val_loss > _val_loss:
            best_val_loss = _val_loss
            best_model = model
    
    return best_model,t_loss,val_loss

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
    with torch.no_grad():
        for imgs, labels_A,labels_B,labels_C,labels_D,labels_E,labels_F,labels_G,labels_H,labels_I,labels_J in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels_A = labels_A.to(device)
            labels_B = labels_B.to(device)
            labels_C = labels_C.to(device)
            labels_D = labels_D.to(device)
            labels_E = labels_E.to(device)
            labels_F = labels_F.to(device)
            labels_G = labels_G.to(device)
            labels_H = labels_H.to(device)
            labels_I = labels_I.to(device)
            labels_J = labels_J.to(device)
            
            A,B,C,D,E,F,G,H,I,J = model(imgs)
            
            loss =  0.1 * criterion(A.to(torch.float32), labels_A.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(B.to(torch.float32), labels_B.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(C.to(torch.float32), labels_C.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(D.to(torch.float32), labels_D.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(E.to(torch.float32), labels_E.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(F.to(torch.float32), labels_F.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(G.to(torch.float32), labels_G.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(H.to(torch.float32), labels_H.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(I.to(torch.float32), labels_I.view([-1, 1]).to(torch.float32)) + \
                    0.1 * criterion(J.to(torch.float32), labels_J.view([-1, 1]).to(torch.float32))
           
            # pred_multi = [A,B,C,D,E,F,G,H,I,J]
            # label_multi = [labels_A,labels_B,labels_C,labels_D,labels_E,labels_F,labels_G,labels_H,labels_I,labels_J]
            # batch_acc_total = 0
            # for arg in range(10):
            #     probs  = pred_multi[arg].cpu().detach().numpy()
            #     labels = label_multi[arg].cpu().detach().numpy()
            #     preds = probs > 0.5
            #     batch_acc_total += (labels == preds).mean()
            
            # val_acc.append(batch_acc_total/10)
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        # _val_acc = np.mean(val_acc)
    
    return _val_loss

In [ ]:
# k=2
# splits=KFold(n_splits=k,shuffle=True,random_state=42)
# foldperf={}

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

best_model_ = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/efficientnet_v2_m-dc08266a.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_m-dc08266a.pth


  0%|          | 0.00/208M [00:00<?, ?B/s]

  0%|          | 0/825 [00:00<?, ?it/s]

  0%|          | 0/207 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.33855] Val Loss : [0.15094]


...

  0%|          | 0/207 [00:00<?, ?it/s]

Epoch [14], Train Loss : [0.01057] Val Loss : [0.05439]


  0%|          | 0/825 [00:00<?, ?it/s]

  0%|          | 0/207 [00:00<?, ?it/s]

Epoch [15], Train Loss : [0.00840] Val Loss : [0.06161]


In [ ]:
# dataset = ConcatDataset([train_dataset, val_dataset])
# history = {'train_loss': [], 'test_loss': []}
# best_model_ = None
# best_val_loss = 99999

# for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

#     print('Fold {}'.format(fold + 1))

#     train_sampler = SubsetRandomSampler(train_idx)
#     test_sampler = SubsetRandomSampler(val_idx)
    
#     train_loader = DataLoader(dataset, batch_size=CFG['BATCH_SIZE'], sampler=train_sampler)
#     test_loader = DataLoader(dataset, batch_size=CFG['BATCH_SIZE'], sampler=test_sampler)
    
#     model = BaseModel()
#     model.to(device)
#     model.eval()
#     optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-4, verbose=True)

#     best_model,train_loss,test_loss = train(model, optimizer, train_loader, test_loader, scheduler, device)
#     if best_val_loss > np.mean(test_loss):
#         best_model_ = best_model
#         best_val_loss = np.mean(test_loss)
#     history['train_loss'].append(train_loss)
#     history['test_loss'].append(test_loss)

In [ ]:
torch.save(best_model_[0].state_dict(), "./efficientnetv2_m.pt")

In [ ]:
# model = BaseModel()
# model.load_state_dict(torch.load('/kaggle/input/efficientnet/effieentnetb0.pt'))
# model.to(device)

In [ ]:
class CustomDatasetTest(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        img_path = os.path.join("./", img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_CUBIC)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
#             image = np.fft.fft2(image)
#             image = np.fft.fftshift(image)
            
        if self.label_list is not None:
            #label = torch.FloatTensor(self.label_list[index])
            
            labels_A = self.label_list["A"][index]
            labels_B = self.label_list["B"][index]
            labels_C = self.label_list["C"][index]
            labels_D = self.label_list["D"][index]
            labels_E = self.label_list["E"][index]
            labels_F = self.label_list["F"][index]
            labels_G = self.label_list["G"][index]
            labels_H = self.label_list["H"][index]
            labels_I = self.label_list["I"][index]
            labels_J = self.label_list["J"][index]
            
            return image,labels_A,labels_B,labels_C,labels_D,labels_E,labels_F,labels_G,labels_H,labels_I,labels_J
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
test_dataset = CustomDatasetTest(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
# train = df[:train_len]
# val = df[train_len:]

In [ ]:
# train_ml_dataset = CustomDataset(train['img_path'].values, train_labels, train_transform)
# train_ml_loader = DataLoader(train_ml_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
# val_ml_dataset = CustomDataset(val['img_path'].values, val_labels, test_transform)
# val_ml_loader = DataLoader(val_ml_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
# def inference(model, test_loader, device):
#     model.to(device)
#     model.eval()
#     predictions = []
#     with torch.no_grad():
#         for imgs in tqdm(iter(test_loader)):
#             imgs = imgs.float().to(device)
            
#             probs = model(imgs)

#             probs  = probs.cpu().detach().numpy()
#             preds = probs > 0.5
#             preds = preds.astype(int)
#             predictions += preds.tolist()
#     return predictions

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions_A = []
    predictions_B = []
    predictions_C = []
    predictions_D = []
    predictions_E = []
    predictions_F = []
    predictions_G = []
    predictions_H = []
    predictions_I = []
    predictions_J = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            A,B,C,D,E,F,G,H,I,J = model(imgs)
            A,B,C,D,E,F,G,H,I,J = A > 0.5, B > 0.5, C > 0.5, D > 0.5, E > 0.5, F > 0.5, G > 0.5, H >0.5, I > 0.5, J > 0.5
            A,B,C,D,E,F,G,H,I,J = A.type(torch.int), B.type(torch.int), C.type(torch.int), D.type(torch.int), E.type(torch.int), F.type(torch.int), G.type(torch.int), H.type(torch.int), I.type(torch.int), J.type(torch.int)
            
            predictions_A += A.cpu().detach().tolist() 
            predictions_B += B.cpu().detach().tolist()
            predictions_C += C.cpu().detach().tolist()
            predictions_D += D.cpu().detach().tolist()
            predictions_E += E.cpu().detach().tolist()
            predictions_F += F.cpu().detach().tolist()
            predictions_G += G.cpu().detach().tolist()
            predictions_H += H.cpu().detach().tolist()
            predictions_I += I.cpu().detach().tolist()
            predictions_J += J.cpu().detach().tolist()
    return predictions_A,predictions_B,predictions_C,predictions_D,predictions_E,predictions_F,predictions_G,predictions_H,predictions_I,predictions_J

In [ ]:
# def inference_train(model, test_loader, device):
#     model.my_dense.to(device)
#     model.eval()
#     predictions = []
#     with torch.no_grad():
#         for imgs,label in tqdm(iter(test_loader)):
#             imgs = imgs.float().to(device)
            
#             preds = model(imgs)

#             preds  = preds.cpu().detach().numpy()

#             predictions += preds.tolist()
#     return predictions

In [ ]:
# def inference(model, test_loader, device):
#     model.my_dense.to(device)
#     model.eval()
#     predictions = []
#     with torch.no_grad():
#         for imgs in tqdm(iter(test_loader)):
#             imgs = imgs.float().to(device)
            
#             preds = model(imgs)

#             preds  = preds.cpu().detach().numpy()

#             predictions += preds.tolist()
#     return predictions

In [ ]:
# train_preds = inference_train(best_model_, train_ml_loader, device)

In [ ]:
# val_preds = inference_train(best_model_, val_ml_loader, device)

In [ ]:
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.linear_model import LogisticRegression
# from xgboost import XGBClassifier

# clf = MultiOutputClassifier(XGBClassifier()).fit(train_preds, train_labels)
# val_preds_ml = clf.predict(val_preds)

In [ ]:
# from sklearn.metrics import accuracy_score

# accuracy_score(val_labels, val_preds_ml)

In [ ]:
preds = inference(best_model_[0], test_loader, device)

  0%|          | 0/46 [00:00<?, ?it/s]

In [ ]:
# preds_ml = clf.predict(preds)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
for idx in range(1,11):
    submit.iloc[:,idx:idx+1] = preds[idx-1]
# submit.iloc[:,1:] = preds_ml
submit.head()

,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,0,1,0,0,0,0,1,0,1,0
1,TEST_00001,0,0,0,0,1,1,0,0,0,0
2,TEST_00002,1,1,0,0,1,1,0,1,0,1
3,TEST_00003,1,1,0,0,1,1,0,1,1,0
4,TEST_00004,0,1,0,0,1,0,0,0,0,0


In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)